In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/glove840b300dtxt/glove.840B.300d.txt


In [7]:
import tensorflow as tf

In [8]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [9]:
df_train1 = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv').fillna('')
#df_train2 = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv').fillna('')
df_val = pd.read_csv('../input/jigsaw-multilingual-toxic-comment-classification/validation.csv').fillna('')

In [10]:
df_train1.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [11]:
df_train1.drop(columns=['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], inplace=True)

In [12]:
df_train1.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [13]:
df_val.head()

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0


**Tokenizing Dataset**

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
df_train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            223549 non-null  object
 1   comment_text  223549 non-null  object
 2   toxic         223549 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.1+ MB


In [31]:
vocab_size = 5000
embedding_dim = 16
max_length = 1500
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [32]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(list(df_train1.comment_text.values) + list(df_val.comment_text.values))
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(df_train1.comment_text.values)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type)

#tokenizer.fit_on_texts(df_val['comment_text'])
testing_sequences = tokenizer.texts_to_sequences(df_val.comment_text.values)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

**Word Embedding**

In [33]:
from tqdm import tqdm

In [34]:
embeddings_index = {}
f = open('../input/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196018it [05:57, 6139.64it/s]

Found 2196017 word vectors.


**Building the Model**

In [35]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 358328/358328 [00:01<00:00, 277811.24it/s]


In [44]:
%%time
with strategy.scope():

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_length,
                     trainable=False))
    model.add(tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(300, dropout=0.6, recurrent_dropout=0.6)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1500, 300)         107498700 
_________________________________________________________________
bidirectional_3 (Bidirection (None, 600)               1442400   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 601       
Total params: 108,941,701
Trainable params: 1,443,001
Non-trainable params: 107,498,700
_________________________________________________________________
CPU times: user 3.01 s, sys: 2.5 s, total: 5.51 s
Wall time: 13.6 s


In [45]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [46]:
# Callback:

cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
data_model = model.fit(training_padded, df_train1.toxic.values, epochs=50, validation_data=(testing_padded, df_val['toxic']),
                      callbacks=[cb], batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/50
437/437 [==============================] - 203s 465ms/step - accuracy: 0.9387 - loss: 0.1746 - val_accuracy: 0.8450 - val_loss: 0.5063
Epoch 2/50
341/437 [======================>.......] - ETA: 42s - accuracy: 0.9493 - loss: 0.1365

In [ ]:
# train data different test data